In [10]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np


In [11]:
# Example small parallel corpus
data = [
    ("hello", "hola"),
    ("how are you", "como estas"),
    ("good morning", "buenos dias"),
    ("thank you", "gracias"),
    ("good night", "buenas noches")
]

In [12]:
# Tokenization
english_texts, spanish_texts = zip(*data)
eng_tokenizer = Tokenizer()
spa_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(english_texts)
spa_tokenizer.fit_on_texts(spanish_texts)

In [13]:
# Convert text to sequences
eng_sequences = eng_tokenizer.texts_to_sequences(english_texts)
spa_sequences = spa_tokenizer.texts_to_sequences(spanish_texts)


In [14]:
# Padding
max_length = max(len(seq) for seq in spa_sequences)
eng_sequences = pad_sequences(eng_sequences, maxlen=max_length, padding='post')
spa_sequences = pad_sequences(spa_sequences, maxlen=max_length, padding='post')

In [17]:
# Define model
embedding_dim = 64
hidden_units = 128

encoder_inputs = tf.keras.Input(shape=(max_length,))
encoder_embedding = Embedding(len(eng_tokenizer.word_index) + 1, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(hidden_units, return_state=True)
_, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = tf.keras.Input(shape=(max_length,))
decoder_embedding = Embedding(len(spa_tokenizer.word_index) + 1, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(len(spa_tokenizer.word_index) + 1, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [18]:
# Compile model
model = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [19]:
# Prepare decoder targets
spa_sequences_output = np.array(spa_sequences).reshape((-1, max_length, 1))

In [20]:
# Train model
model.fit([eng_sequences, spa_sequences], spa_sequences_output, epochs=100, verbose=1)

Epoch 1/100
1/1 [==============================] - 5s 5s/step - loss: 2.1983 - accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 24ms/step - loss: 2.1913 - accuracy: 0.2000
Epoch 3/100
1/1 [==============================] - 0s 20ms/step - loss: 2.1842 - accuracy: 0.2000
Epoch 4/100
1/1 [==============================] - 0s 31ms/step - loss: 2.1771 - accuracy: 0.2000
Epoch 5/100
1/1 [==============================] - 0s 19ms/step - loss: 2.1698 - accuracy: 0.2000
Epoch 6/100
1/1 [==============================] - 0s 23ms/step - loss: 2.1623 - accuracy: 0.2000
Epoch 7/100
1/1 [==============================] - 0s 35ms/step - loss: 2.1545 - accuracy: 0.2000
Epoch 8/100
1/1 [==============================] - 0s 19ms/step - loss: 2.1464 - accuracy: 0.2000
Epoch 9/100
1/1 [==============================] - 0s 20ms/step - loss: 2.1378 - accuracy: 0.2000
Epoch 10/100
1/1 [==============================] - 0s 17ms/step - loss: 2.1287 - accuracy: 0.2000
Epoch 11/100
1/1 

In [21]:
# Translation function
def translate(sentence):
    sequence = eng_tokenizer.texts_to_sequences([sentence])
    sequence = pad_sequences(sequence, maxlen=max_length, padding='post')
    prediction = model.predict([sequence, sequence])
    predicted_words = [spa_tokenizer.index_word.get(np.argmax(word)) for word in prediction[0]]
    return " ".join([w for w in predicted_words if w])

In [22]:
# Example translation
print("Translation:", translate("hello"))

1/1 [==============================] - 1s 802ms/step
Translation: hola
